<a href="https://colab.research.google.com/github/arnepeine/healthdata-tools/blob/master/Predicting_Journal_Title_by_Abstract_with_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim
!pip install bs4
!pip install lxml
!pip install six
!pip install nltk
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
#drive.mount('/content/gdrive')
from nltk.corpus import stopwords
#nltk.download()

    100% |████████████████████████████████| 23.6MB 1.4MB/s 
    100% |████████████████████████████████| 1.4MB 13.9MB/s 
    100% |████████████████████████████████| 133kB 27.4MB/s 
    100% |████████████████████████████████| 61kB 21.6MB/s 
    100% |████████████████████████████████| 4.8MB 6.9MB/s 
    100% |████████████████████████████████| 552kB 25.2MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - \ done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
  Running setup.py bdist_wheel for bs4 ... - done
  Stored in directory: /root/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
    100% |████████████████████████████████| 5.8MB 5.4MB/s 


In [9]:
df = pd.read_csv('http://4points.de/train.txt', sep="\t")
df.head(20)

#print("Word set size")
#df['content'].apply(lambda x: len(x.split(' '))).sum()

#cnt_pro = df['title'].value_counts()
#plt.figure(figsize=(12,4))
#sns.barplot(cnt_pro.index, cnt_pro.values, alpha=0.8)
#plt.ylabel('Number of Occurrences', fontsize=12)
#plt.xlabel('Journal', fontsize=12)
#plt.xticks(rotation=90)
#plt.show();# 

,###24293578
OBJECTIVE,To investigate the efficacy of 6 weeks of dail...
METHODS,A total of 125 patients with primary knee OA w...
METHODS,Outcome measures included pain reduction and i...
METHODS,Pain was assessed using the visual analog pain...
METHODS,Secondary outcome measures included the Wester...
METHODS,"Serum levels of interleukin 1 ( IL-1 ) , IL-6 ..."
RESULTS,There was a clinically relevant reduction in t...
RESULTS,The mean difference between treatment arms ( 9...
RESULTS,"Further , there was a clinically relevant redu..."
RESULTS,These differences remained significant at 12 w...


In [0]:
# here comes the appropriate index matching #### as index per row

In [0]:
# here comes the gensim topic model

In [10]:
# Print example from dataset
def print_complaint(index):
    example = df[df.index == index][['content', 'title']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Journal:', example[1])
print_complaint(145)

KeyError: ignored

In [0]:

# Preprocessing steps for doc2vec

def cleanText(text):
    text = BeautifulSoup(text).text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text
df['content'] = df['content'].apply(cleanText)



In [0]:
train, test = train_test_split(df, test_size=0.3, random_state=42)

from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['content']), tags=[r.title]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['content']), tags=[r.title]), axis=1)

In [38]:
train_tagged.values[30]

import multiprocessing
cores = multiprocessing.cpu_count()

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=30)
model_dbow.alpha -= 0.002
model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 142/142 [00:00<00:00, 304183.44it/s]


In [36]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors
  
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.967741935483871
Testing F1 score: 0.967404133773475
